In [1]:
import findspark
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv
from pyspark.sql.functions import to_date

load_dotenv()

True

In [ ]:
# Check if the data is comma or | seperated 
with open("../../raw data/Sample Superstore - Orders.csv", "r", encoding="cp1252") as f:
    for _ in range(5):
        print(f.readline())

Order ID,Customer ID,Customer Name,Region,State or Province,City,Postal Code,Customer Segment,Order Priority,Ship Mode,Product Category,Product Sub-Category,Product Container,Product Name,Order Date,Ship Date,Discount,Unit Price,Discount Amount,Discount Price,Shipping Cost,Quantity

88525,2,Janice Fletcher,Central,Illinois,Addison,60101,Corporate,Not Specified,Regular Air,Office Supplies,Labels,Small Box,Avery 49,5/28/2012,5/30/2012,0.01,2.88,0.03,2.85,0.5,2

88522,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,High,Express Air,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,7/7/2010,7/8/2010,0.01,2.84,0.03,2.81,0.93,4

88523,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,Not Specified,Express Air,Office Supplies,Paper,Small Box,Xerox 1968,7/27/2011,7/28/2011,0.03,6.68,0.20,6.48,6.15,7

88523,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,Not Specified,Regular Air,Office Supplies,"Scissors, Rulers and Trimmers

In [ ]:
findspark.init()

spark = SparkSession.builder \
    .appName("ETL Pipeline") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.extraClassPath", r"C:\Drivers\sqljdbc_12.10.0.0_enu\sqljdbc_12.10\enu\jars\mssql-jdbc-12.10.0.jre11.jar") \
    .getOrCreate()

# Read CSV
df = spark.read.csv(
    "../../raw data/Sample Superstore - Orders.csv",
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True  
)

returns = spark.read.csv(
    "../../raw data/Sample Superstore - Returns.csv", 
    header=True, 
    inferSchema=True, 
    sep=",",
    quote='"',
    escape='"',
    multiLine=True
)

spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

Order ID,Customer ID,Customer Name,Region,State or Province,City,Postal Code,Customer Segment,Order Priority,Ship Mode,Product Category,Product Sub-Category,Product Container,Product Name,Order Date,Ship Date,Discount,Unit Price,Discount Amount,Discount Price,Shipping Cost,Quantity
88525,2,Janice Fletcher,Central,Illinois,Addison,60101,Corporate,Not Specified,Regular Air,Office Supplies,Labels,Small Box,Avery 49,5/28/2012,5/30/2012,0.01,2.88,0.03,2.85,0.5,2
88522,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,High,Express Air,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Ac...,7/7/2010,7/8/2010,0.01,2.84,0.03,2.81,0.93,4
88523,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,Not Specified,Express Air,Office Supplies,Paper,Small Box,Xerox 1968,7/27/2011,7/28/2011,0.03,6.68,0.2,6.48,6.15,7
88523,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,Not Specified,Regular Air,Office Supplies,"Scissors, Rulers ...",Small Pack,Acme� Preferred S...,7/27/2011,7/28/2011,0.01,5.68,0.06,5.62,3.6,7
88523,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,Not Specified,Express Air,Technology,Telephones and Co...,Small Box,V70,7/27/2011,7/27/2011,0.0,205.99,0.0,205.99,2.5,8
88524,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,Medium,Express Air,Office Supplies,Paper,Small Box,Xerox 194,11/9/2011,11/11/2011,0.09,55.48,4.99,50.49,14.3,37
88526,3,Bonnie Potter,West,Washington,Anacortes,98221,Corporate,Low,Delivery Truck,Technology,Office Machines,Jumbo Drum,Canon S750 Color ...,7/1/2013,7/8/2013,0.08,120.97,9.68,111.29,26.3,12
90193,5,Ronnie Proctor,West,California,San Gabriel,91776,Home Office,Not Specified,Delivery Truck,Furniture,Chairs & Chairmats,Jumbo Drum,Global Troy� Exec...,12/13/2010,12/15/2010,0.02,500.98,10.02,490.96,26.0,12
90197,5,Ronnie Proctor,West,California,San Gabriel,91776,Home Office,Low,Regular Air,Office Supplies,Paper,Small Box,Xerox 1930,5/12/2012,5/21/2012,0.08,6.48,0.52,5.96,6.81,18
90194,6,Dwight Hwang,West,California,San Jose,95123,Home Office,High,Regular Air,Office Supplies,Appliances,Small Box,Kensington 6 Outl...,5/26/2011,5/26/2011,0.01,90.24,0.9,89.34,0.99,16


In [4]:
returns

Order ID,Status
65,Returned
612,Returned
614,Returned
678,Returned
710,Returned
740,Returned
775,Returned
833,Returned
902,Returned
3300,Returned


In [5]:
df.describe().show()

+-------+-----------------+------------------+------------------+-------+-----------------+--------+-----------------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+----------+---------+--------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|         Order ID|       Customer ID|     Customer Name| Region|State or Province|    City|      Postal Code|Customer Segment|Order Priority|     Ship Mode|Product Category|Product Sub-Category|Product Container|        Product Name|Order Date|Ship Date|            Discount|        Unit Price|   Discount Amount|   Discount Price|     Shipping Cost|          Quantity|
+-------+-----------------+------------------+------------------+-------+-----------------+--------+-----------------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+

| COLUMN                    | NOTE                                                                  | SOLVABLE | PROPOSED PROCEDURE                                                                                                                     |
| ------------------------- | --------------------------------------------------------------------- | -------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| `Product Name`            | Mean and stddev still exist → likely numeric leakage into column      | Yes        | Check if numeric fields (e.g., `Discount Price`) spilled into this column; investigate via `.select("Product Name").distinct().show()` |
| `Product Name`            | Some values are wrapped in quotes (e.g. `"While you Were O..."`)      | Yes        | Use `.withColumn()` + regex/trim to clean extra quotes from string fields                                                              |
| `Unit Price`              | Max value is **6783.02** (vs. mean \~88.3) → extreme outliers present | Yes        | Identify and flag records with `Unit Price > 1000` as potential outliers                                                               |
| `Discount`                | Range from `0.0` to `0.25` seems valid but needs verification         | Yes        | Confirm all values fall within 0–1; filter out invalid discount entries                                                                |
| `Order Date`, `Ship Date` | Still treated as strings (no mean/stddev)                             | Yes        | Convert to date type using `to_date()` and compute `delivery_delay`                                                                    |
| `Shipping Cost`           | Outliers exist (max = 164.73 vs. mean = 12.79)                        | Yes        | Flag and investigate rows where `Shipping Cost > 100`                                                                                  |                                                           |
| `Postal Code`             | Numeric but actually categorical; leading zeros may be lost           | Yes        | Cast as string to preserve format and leading digits                                                                                   |


### Standardize Product Name

In [6]:
from pyspark.sql.functions import col, lower, trim, regexp_replace

df_cleaned = df.withColumn(
    "Product Name",
    regexp_replace(trim(lower(col("Product Name"))), "[^a-z0-9 ]", "")
)


In [7]:
df_cleaned.show()

+--------+-----------+--------------------+-------+-----------------+-----------+-----------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+
|Order ID|Customer ID|       Customer Name| Region|State or Province|       City|Postal Code|Customer Segment|Order Priority|     Ship Mode|Product Category|Product Sub-Category|Product Container|        Product Name|Order Date| Ship Date|Discount|Unit Price|Discount Amount|Discount Price|Shipping Cost|Quantity|
+--------+-----------+--------------------+-------+-----------------+-----------+-----------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+
|   88525|          2|     Janice Fletcher|Central|       

In [8]:
df_cleaned.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Customer ID: integer (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- State or Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Product Sub-Category: string (nullable = true)
 |-- Product Container: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Discount Amount: double (nullable = true)
 |-- Discount Price: double (nullable = true)
 |-- Shipping Cost: double (nullable = true)
 |-- Quantity: integer (nullable = true)



In [9]:
from pyspark.sql.types import StringType

df_cleaned = df_cleaned \
    .withColumn("Postal Code", col("Postal Code").cast(StringType())) \
    .withColumn("Order Date", to_date(col("Order Date"), "M/d/yyyy")) \
    .withColumn("Ship Date", to_date(col("Ship Date"), "M/d/yyyy"))


In [10]:
df_cleaned.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Customer ID: integer (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- State or Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Product Sub-Category: string (nullable = true)
 |-- Product Container: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Discount Amount: double (nullable = true)
 |-- Discount Price: double (nullable = true)
 |-- Shipping Cost: double (nullable = true)
 |-- Quantity: integer (nullable = true)



In [11]:
df_cleaned.show()

+--------+-----------+--------------------+-------+-----------------+-----------+-----------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+
|Order ID|Customer ID|       Customer Name| Region|State or Province|       City|Postal Code|Customer Segment|Order Priority|     Ship Mode|Product Category|Product Sub-Category|Product Container|        Product Name|Order Date| Ship Date|Discount|Unit Price|Discount Amount|Discount Price|Shipping Cost|Quantity|
+--------+-----------+--------------------+-------+-----------------+-----------+-----------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+
|   88525|          2|     Janice Fletcher|Central|       

In [12]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

@pandas_udf('string')
def remove_middle_name(customer_names: pd.Series) -> pd.Series:
    def format_name(name):
        parts = name.strip().split()
        if len(parts) == 1:
            return parts[0]
        elif len(parts) == 2:
            return f"{parts[0]} {parts[1]}"
        else:
            return f"{parts[0]} {parts[-1]}"
    
    return customer_names.apply(format_name)


In [13]:
df_cleaned.filter(col("Discount") > 1).show()

+--------+-----------+-------------+------+-----------------+----+-----------+----------------+--------------+---------+----------------+--------------------+-----------------+------------+----------+---------+--------+----------+---------------+--------------+-------------+--------+
|Order ID|Customer ID|Customer Name|Region|State or Province|City|Postal Code|Customer Segment|Order Priority|Ship Mode|Product Category|Product Sub-Category|Product Container|Product Name|Order Date|Ship Date|Discount|Unit Price|Discount Amount|Discount Price|Shipping Cost|Quantity|
+--------+-----------+-------------+------+-----------------+----+-----------+----------------+--------------+---------+----------------+--------------------+-----------------+------------+----------+---------+--------+----------+---------------+--------------+-------------+--------+
+--------+-----------+-------------+------+-----------------+----+-----------+----------------+--------------+---------+----------------+--------

In [14]:
df_cleaned.filter(col("Shipping Cost") > 100).show()

+--------+-----------+--------------------+-------+-----------------+---------------+-----------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+
|Order ID|Customer ID|       Customer Name| Region|State or Province|           City|Postal Code|Customer Segment|Order Priority|     Ship Mode|Product Category|Product Sub-Category|Product Container|        Product Name|Order Date| Ship Date|Discount|Unit Price|Discount Amount|Discount Price|Shipping Cost|Quantity|
+--------+-----------+--------------------+-------+-----------------+---------------+-----------+----------------+--------------+--------------+----------------+--------------------+-----------------+--------------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+
|   90436|        194|       Tammy Goldman|   

shipping cost is per product being shipped

In [15]:
from functools import reduce
import re

# Function to convert to snake_case
def to_snake_case(name):
    name = re.sub(r'[\s\-]+', '_', name)  # Replace spaces/dashes with underscores
    name = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', name)  # Convert camelCase
    return name.lower()

# Original and new column names
oldColumns = df_cleaned.schema.names
newColumns = [to_snake_case(name) for name in oldColumns]

# Rename columns using reduce
df_cleaned = reduce(
    lambda df, idx: df.withColumnRenamed(oldColumns[idx], newColumns[idx]),
    range(len(oldColumns)),
    df_cleaned
)


In [16]:
# Original and new column names
return_old_columns = returns.schema.names
return_new_columns = [to_snake_case(name) for name in return_old_columns]

# Rename columns using reduce
returns = reduce(
    lambda df, idx: df.withColumnRenamed(return_old_columns[idx], return_new_columns[idx]),
    range(len(return_old_columns)),
    returns
)


In [17]:
try:
    jdbc_url = (
    "jdbc:sqlserver://DESKTOP-2ETDICD;databaseName=superstore_etl;"
    "encrypt=true;trustServerCertificate=true;"
    )

    df_cleaned.write \
      .format("jdbc") \
      .option("url", jdbc_url) \
      .option("dbtable", "dbo.staging_orders") \
      .option("user", os.getenv('DB_USER')) \
      .option("password", os.getenv('DB_PASSWORD')) \
      .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      .mode("overwrite") \
      .save()
    
    returns.write \
      .format("jdbc") \
      .option("url", jdbc_url) \
      .option("dbtable", "dbo.customer_returns") \
      .option("user", os.getenv('DB_USER')) \
      .option("password", os.getenv('DB_PASSWORD')) \
      .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      .mode("overwrite") \
      .save()
    
except Exception as err:
  print(err)